<br>

## Initial Setup

---

In [12]:
# Import the libraries
import os
import pandas as pd
import importlib
import langchain
import langchain_community
from langchain.document_loaders import *
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.chains import *
from langchain.text_splitter import * 
from langchain_community.embeddings import * 
from langchain_community.vectorstores import *

In [2]:
# Change the path to root directory
os.chdir('..') 

# Print the current working directory
os.getcwd()  

'/home/predator/Desktop/code base/enigma-code'

In [3]:
# Import the local modules
import enigma_code
import enigma_code as ec
from enigma_code import data
from enigma_code import vectorstore

In [4]:
import importlib

# Assuming enigma_code was already imported as shown in the active selection
importlib.reload(enigma_code)

# If you have imported submodules or used aliases, you might need to reload those as well
importlib.reload(ec)
importlib.reload(data)
importlib.reload(vectorstore)

<module 'enigma_code.vectorstore' from '/home/predator/Desktop/code base/enigma-code/enigma_code/vectorstore.py'>

In [5]:
# Load .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

True

<br>

## Load Dataset

---

In this section, we will load the dataset from various sources.

#### Local Resources

In [6]:
# Instantiat the data handler
data_handler = ec.data.DataHandler()

In [7]:
# Load a text dataset
dataset = data_handler.load_dataset("datasets/rules_and_regulations.txt")
dataset

[Document(metadata={'source': 'datasets/rules_and_regulations.txt'}, page_content="1. Guests will be charged per night.\n2. Check-in time is 2:00 p.m. on the day of arrival and the check-out time is 12:00 p.m. on the day of departure.\n3. If upon check-in the guest does not specify the duration of his or her stay the hotel will presume it is for one night.\n4. Guest must show up before 2 p.m., and then the booking will be cancelled if there is no contact from guest.\n5. Only 2 guests are allowed to stay in a room. Children older than 7 years old or the third guest must pay 500 Thai baht for extra bed.\n6. Baby cot can be provided to a child under 2 years old upon request.\n7. Bills must be settled by cash or valid credit card, personal cheques are not accepted.\n8. Guests wishing to check out later than 2:00 p.m. are kindly requested to ask for permission from our reception before 12:00 p.m. (subject to room availability, otherwise it will be charged for one additional night.\n9. Smoki

In [8]:
# Chunk the dataset 
chunks = data_handler.chunk_dataset(documents=dataset, chunk_type="newline", chunk_size=None, chunk_overlap=None)
chunks

[Document(page_content='1. Guests will be charged per night.'),
 Document(page_content='2. Check-in time is 2:00 p.m. on the day of arrival and the check-out time is 12:00 p.m. on the day of departure.'),
 Document(page_content='3. If upon check-in the guest does not specify the duration of his or her stay the hotel will presume it is for one night.'),
 Document(page_content='4. Guest must show up before 2 p.m., and then the booking will be cancelled if there is no contact from guest.'),
 Document(page_content='5. Only 2 guests are allowed to stay in a room. Children older than 7 years old or the third guest must pay 500 Thai baht for extra bed.'),
 Document(page_content='6. Baby cot can be provided to a child under 2 years old upon request.'),
 Document(page_content='7. Bills must be settled by cash or valid credit card, personal cheques are not accepted.'),
 Document(page_content='8. Guests wishing to check out later than 2:00 p.m. are kindly requested to ask for permission from our 

In [9]:
len(dataset)

1

In [10]:
len(chunks)

34

In [11]:
# TODO: Add functions for cleaning (e.g., beutiful soup, regex, etc.)
# TODO: Add functions for preprocessing (e.g., tokenization, lemmatization, etc.)

#### Databases

In [10]:
# Instantiate the database query tool
db_query_tool = ec.data.DatabaseQueryTool(
    database_type="mysql",
    username=os.environ.get("MYSQL_USERNAME"),
    password=os.environ.get("MYSQL_PASSWORD"),
    database="company_x",
    host=os.environ.get("MYSQL_HOST"),
    port=os.environ.get("MYSQL_PORT")
)

In [11]:
# List all available tables
db_query_tool.run_sql_query("SHOW TABLES")

[('chat_history',), ('company_knowledge_base',), ('faq',), ('users',)]

In [12]:
# Load the tables
chat_history_df = pd.read_sql_query(sql="SELECT * FROM company_x.chat_history", con=db_query_tool.engine)
company_knowledge_base_df = pd.read_sql_query(sql="SELECT * FROM company_x.company_knowledge_base", con=db_query_tool.engine)
faq_df = pd.read_sql_query(sql="SELECT * FROM company_x.faq", con=db_query_tool.engine)
users_df = pd.read_sql_query(sql="SELECT * FROM company_x.users", con=db_query_tool.engine)

# Display the chat history
display("Chat History: ", chat_history_df.head())
display("Company Knowledge Base: ", company_knowledge_base_df.head())
display("FAQ: ", faq_df.head())
display("Users: ", users_df.head())

'Chat History: '

,session_id,user_id,question,response,chat_date
0,1,1,How do I change my password?,You can change your password from the settings...,2024-07-15 20:00:41
1,2,2,Where can I find the refund policy?,The refund policy can be found under the Terms...,2024-07-15 20:00:41
2,3,3,Can I update my email address?,"Yes, you can update your email in account sett...",2024-07-15 20:00:41
3,4,4,What are the support hours?,Our support is available 24/7.,2024-07-15 20:00:41
4,5,5,How to track my order?,You can track your order through the Orders se...,2024-07-15 20:00:41


'Company Knowledge Base: '

,kb_id,category,sub_category,text_content,created_date
0,1,Privacy Policy,General,Our privacy policy outlines how we handle your...,2024-07-15 20:00:41
1,2,Terms of Service,Usage,Terms of service governing the use of our serv...,2024-07-15 20:00:41
2,3,Refund Policy,E-commerce,Our refund policy for purchases made on our pl...,2024-07-15 20:00:41
3,4,Legal,Compliance,Legal compliances that our company adheres to.,2024-07-15 20:00:41
4,5,Refund Policy,Subscriptions,How refunds are handled for subscription servi...,2024-07-15 20:00:41


'FAQ: '

,faq_id,question,answer,category,sub_category,created_at
0,1,How do I reset my password?,You can reset your password by going to settings.,Account,Settings,2024-07-15 20:00:41
1,2,Where can I view my transactions?,Your transactions are available under the Hist...,Account,History,2024-07-15 20:00:41
2,3,How do I contact support?,Support can be reached via the Contact Us page.,Support,General,2024-07-15 20:00:41
3,4,What is the refund policy?,Our refund policy is outlined in the Terms of ...,Purchase,Refunds,2024-07-15 20:00:41
4,5,Do you offer discounts?,Current discount offers are listed on our Prom...,Sales,Promotions,2024-07-15 20:00:41


'Users: '

,user_id,first_name,last_name,email,password,created_at,role
0,1,Alice,Johnson,alice.johnson@email.com,password123,2024-07-15 20:00:41,user
1,2,Bob,Smith,bob.smith@email.com,password123,2024-07-15 20:00:41,admin
2,3,Carol,Taylor,carol.taylor@email.com,password123,2024-07-15 20:00:41,user
3,4,David,Brown,david.brown@email.com,password123,2024-07-15 20:00:41,user
4,5,Emma,Davis,emma.davis@email.com,password123,2024-07-15 20:00:41,user


<br>

## RAG

---

Transform the knowledge base dataset into embedding vectors and save them in the vector store for use in the RAG process.

#### Vectorstore

In [14]:
# Initialize VectorStoreHandler
vs = ec.vectorstore.VectorStoreManager(
    vectorstore_name="pinecone",
    embedding_model="openai",
    index_name="index-name"
)

/home/predator/.cache/pypoetry/virtualenvs/enigma-code-0aWLlfTL-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:
# Load the vectorstore
vectorstore = vs.load_vectorstore()

In [14]:
# Add documents

# Test adding documents
documents = [
    Document(page_content="This is a test document about AI.", metadata={"source": "test1"}),
    Document(page_content="Vector databases are useful for similarity search.", metadata={"source": "test2"}),
    Document(page_content="LangChain is a framework for developing applications powered by language models.", metadata={"source": "test3"})
]

# Add documents to the vectorstore
vectorstore.add_documents(documents=documents)

['c1c65162-e1bc-47e0-9b03-19a5556b431d',
 'bc78a297-ed66-480d-8b20-dc2bbbaad53f',
 '537423aa-e8c9-40e9-bee8-f63ef3736cc6']

In [23]:
# Seach for similar documents
query = "This is a test document about AI."
vectorstore.search(query=query, search_type="similarity")

[Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.')]

In [17]:
vectorstore

#### RAG

In [13]:
class RAG:
    
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
        self.setup_chat_chain()
        self.chat_history = []        

    def setup_chat_chain(self):
        self.chat_chain = langchain.chains.ConversationalRetrievalChain.from_llm(
            self.llm, 
            self.vectorstore.as_retriever(), 
            return_source_documents=True
        )

    def chat_query(self, question):
        result = self.chat_chain({"question": question, "chat_history": self.chat_history})
        self.chat_history.append((question, result["answer"]))
        return result['answer']

In [14]:
# LLM
llm = langchain_community.llms.Ollama(model="llama3") #temperature=temperature, top_p=top_p)

In [15]:
# Vectorstore
loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
docs = loader.load()
text_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)
vectorstore = langchain_community.vectorstores.FAISS.from_documents(all_splits, langchain_community.embeddings.HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [16]:
# RAG
rag = RAG(llm, vectorstore)

In [17]:
# Ask a question based on the vectorstore
rag.chat_query("What's new with Llama 3?")

"According to the context, what's new with Llama 3 is the introduction of 4 new open LLM models by Meta based on the Llama 2 architecture. The release also comes with a new tokenizer that expands the vocabulary size from 32K tokens in Llama 2 to 128,256 tokens in Llama 3."

In [18]:
# Follow-up question (it has knowledge of the previous question)
rag.chat_query("Based on what architecture?")

'According to the text, Llama 3 is based on the Llama 2 architecture.'

<br>

## Fine-Tune Model

---

In this section, we will fine-tune the base model using the chat dataset.

#### Data Prepration

##### Fine-Tunning

<br>

## Agentic Tools

---

<br>

## Prompt Engineering

---